# Main Library

In [1]:
!pip install transformers -U
!pip install accelerate -U
!pip install datasets -U

In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
# !pip install torch

In [3]:
from datasets import load_dataset, Dataset
import torch

In [4]:
model_name = 'distilbert-base-uncased'

# Reading Data

In [5]:
data = pd.read_csv(r'/content/SMSSpamCollection.txt', sep='\t', names=['label', 'text'])
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
data.isnull().sum()

label    0
text     0
dtype: int64

In [7]:
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])
print(le.classes_)

['ham' 'spam']


# Ham --> 0
# Spam --> 1

# Splitting Data Into Trianing & Testing

In [8]:
dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.2)

# Make Tokenization using Transformer

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4457 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

# Make Transformer Arguments

In [14]:
training_arg = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

In [16]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Define a function to compute metrics
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Building Model

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 4457
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1115
    })
})

In [19]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_arg,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss


# Make Prediction

In [ ]:
trainer.evaluta(testing_data)

In [ ]:
trainer.predict(testing_data)

In [ ]:
prediction = trainer.predict(testing_data)[1]
prediction

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, prediction)
cm

# Save Model

In [ ]:
trainer.save_model('spam-ham_model')